In [1]:
import requests
from bs4 import BeautifulSoup
import json
from pprint import pprint
import numpy as np
import pandas as pd

In [2]:
res = requests.get('http://openapi.seoul.go.kr:8088/5058644a58646c733132324e64486957/json/bikeList/1/1000/')
data = json.loads(res.content)
df_1 = pd.DataFrame.from_dict(data['rentBikeStatus']['row'])


res_2 = requests.get('http://openapi.seoul.go.kr:8088/5058644a58646c733132324e64486957/json/bikeList/1001/2000')
data_2 = json.loads(res_2.content)
df_2 = pd.DataFrame.from_dict(data_2['rentBikeStatus']['row'])


fin_data = pd.concat([df_1, df_2],ignore_index=True)

In [3]:
fin_data['station_No']=fin_data['stationName'].apply(lambda e: e.split('.')[0])
fin_data['station_Name']=fin_data['stationName'].apply(lambda e: e.split('.')[1])

In [4]:
fin_data = fin_data.drop(columns=['stationName'])

In [5]:
arr = pd.read_csv(r'C:\Users\19641\OSS_Project\opensourceproject\bicycle\using_No.csv',names = ["station_No"])

In [6]:
arr['station_No'] = arr['station_No'].astype(str)

In [7]:
fin_data = fin_data[fin_data["station_No"].isin(arr['station_No'])]
fin_data = fin_data.reset_index(drop=True)

In [8]:
dt_in = pd.read_csv(r"C:\Users\19641\OSS_Project\opensourceproject\bicycle\data_in.csv")
dt_out = pd.read_csv(r"C:\Users\19641\OSS_Project\opensourceproject\bicycle\data_out.csv")

In [9]:
data = pd.merge(dt_in, dt_out, on='station_id')

In [10]:
outcome_1=pd.merge(fin_data, data, how='outer', left_index=True, right_index=True)

In [11]:
outcome_1 = outcome_1.drop(columns=['station_id'])

In [12]:
dt = pd.read_csv(r'C:\Users\19641\OSS_Project\opensourceproject\bicycle\공공자전거 대여소 정보_201905.csv', engine='python')

In [13]:
dt= dt[dt["대여소번호"].isin(arr['station_No'])]
dt = dt.reset_index(drop=True)
dt = dt.drop(columns=['대여소명', '위도', '경도','거치대수'])

In [14]:
cols = dt.columns.tolist()

cols = cols[-1:] + cols[:-1]

cols

dt = dt[cols]

In [15]:
final_out = pd.concat([dt, outcome_1], axis=1)
final_out = final_out.drop(columns='station_No')
final_out = final_out.rename(columns={'대여소번호':'station_Num'})

In [16]:
station_flag = final_out['station_Name'].str.contains("역 ")
final_out['station_flag'] = station_flag

In [17]:
searchfor = ['학교', '대 ']
school_flag = final_out['station_Name'].str.contains('|'.join(searchfor))
final_out['school_flag'] = school_flag

In [18]:
final_out.to_csv('final_out.csv',encoding='utf-8-sig',index=False)

In [19]:
final_out

,station_Num,구분,parkingBikeTotCnt,rackTotCnt,shared,stationId,stationLatitude,stationLongitude,station_Name,in 0:00 ~ 1:00,...,out 16:00 ~ 17:00,out 17:00 ~ 18:00,out 18:00 ~ 19:00,out 19:00 ~ 20:00,out 20:00 ~ 21:00,out 21:00 ~ 22:00,out 22:00 ~ 23:00,out 23:00 ~ 24:00,station_flag,school_flag
0,101,마포구,2,5,40,ST-3,37.54956055,126.90575409,(구)합정동 주민센터,0.283641,...,0.323041,0.517742,0.572350,0.390783,0.288710,0.211290,0.171889,0.249309,False,False
1,102,마포구,19,20,95,ST-4,37.55564880,126.91062927,망원역 1번출구 앞,1.217742,...,1.507604,1.638249,1.704608,1.570507,1.276959,1.496083,1.551382,1.216129,True,False
2,103,마포구,3,14,21,ST-5,37.55495071,126.91083527,망원역 2번출구 앞,0.879263,...,1.330415,1.252535,1.440323,1.371429,0.997926,0.955530,0.790783,0.731797,True,False
3,104,마포구,1,13,8,ST-6,37.55062866,126.91498566,합정역 1번출구 앞,0.991705,...,1.451152,1.452995,1.887558,1.697005,1.243088,1.284562,1.163594,1.006221,True,False
4,105,마포구,0,5,0,ST-7,37.55000687,126.91482544,합정역 5번출구 앞,0.773963,...,0.807143,0.787788,0.869355,0.827880,0.675576,0.780645,0.767512,0.780415,True,False
5,106,마포구,0,10,0,ST-8,37.54864502,126.91282654,합정역 7번출구 앞,1.375115,...,1.928571,2.306452,2.458756,1.654839,1.553917,1.707604,1.713594,1.459677,True,False
6,107,마포구,5,5,100,ST-9,37.55751038,126.91850281,신한은행 서교동금융센터점 앞,0.969585,...,1.339171,1.792627,2.371198,1.709908,1.571198,1.438479,1.482258,1.202304,False,False
7,108,마포구,4,10,40,ST-10,37.55274582,126.91861725,서교동 사거리,0.792627,...,1.168203,1.107834,1.355300,1.394240,0.930415,1.074424,1.146544,1.102995,False,False
8,109,마포구,3,10,30,ST-11,37.54769135,126.91998291,제일빌딩 앞,0.960829,...,0.977419,0.860138,0.812673,0.887788,0.903687,0.761521,0.915899,0.766820,False,False
9,110,서대문구,3,20,15,ST-13,37.56819916,126.91784668,사천교,0.635945,...,0.547926,0.707373,0.950230,0.655760,0.403687,0.514977,0.522120,0.458986,False,False
